In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string 
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [2]:
url = 'http://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'

In [3]:
dataset = tf.keras.utils.get_file('stack_overflow_16k.tar.gz', url, untar=True, cache_dir='.', cache_subdir='.')
dataset_dir = os.path.join(os.path.dirname(dataset), 'stack_overflow')

6053888/6053168 [==============================] - 0s 0us/step


In [8]:
sample_file = os.path.join('train/', 'python/1500.txt')

In [9]:
with open(sample_file) as f:
  print(f.read())

"using blank to encapsulate part of a string after 3 commas i am trying to create a blank script that adds quotations around part of a string, after 3 commas..so if the input data looks like this:..1234,1,1/1/2010,this is a test. one, two, three....i want blank to convert the string to:..1234,1,1/1/2010,""this is a test. one, two, three.""...the quotes will always need to be added after 3 commas..i am using blank 3.1.2 and have the following so far:..i_file=open(""input.csv"",""r"").o_file=open(""output.csv"",""w"")..for line in i_file:.        tokens=line.split("","").        count=0.        new_line="""".        for element in tokens:.                if count = ""3"":.                        new_line = new_line + '""' + element + '""'.                        break.                else:.                        new_line = new_line + element + "","".                        count=count+1..        o_file.write(new_line + ""n"").        print(line, "" -&gt; "", new_line)..i_file.close().o_

In [10]:
ls train/

csharp/  java/  javascript/  python/


In [11]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory('train/',
                                                                  batch_size=batch_size,
                                                                  seed=seed,
                                                                  validation_split=0.2,
                                                                  subset='training')

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [13]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(3):
    print('Question: ', text_batch.numpy()[i])
    print('Label: ', label_batch.numpy()[i])

Question:  b'"blank8 why is my solution faster than the neat solution? (hackerrank chocolate feast) edit: simplified my solution..edit: removed opinion based secondary question...background: atarted learning blank a week or two ago using hackerranks problems as exercises and stackoverflow search + google as my teacher, i\'ve had some limited experience learning other languages...i did the exercise my own ""noobish learner way"" which i can\'t help but feel is a ""botched job"" when i see ""neat &amp; short"" solutions...however, when submitting both solutions one after another a couple of times i found the ""neat"" solution was quite a bit slower. ..i vaguely remember something about % operations being costly, is mine faster because of no % operations or is there more to it than just that?..exercise: https://www.hackerrank.com/challenges/chocolate-feast..neat solution from discussion:..import blank.io.*;.import blank.util.*;..public class solution {.    static int cc; .    public stati

In [15]:
for i in range(4):
  print(f'Label {i} corresponds to {raw_train_ds.class_names[i]}')

Label 0 corresponds to csharp
Label 1 corresponds to java
Label 2 corresponds to javascript
Label 3 corresponds to python


In [16]:
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory('train/',
                                                                  batch_size=batch_size,
                                                                  seed=seed,
                                                                  validation_split=0.2,
                                                                  subset='validation')

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [28]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory('test/',
                                                                 batch_size=batch_size)

Found 8000 files belonging to 4 classes.


In [21]:
def custom_standarization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase, '[%s]' % re.escape(string.punctuation), ' ')

In [22]:
max_features = 10000
sequence_length = 250

vectorize_layer = TextVectorization(standardize=custom_standarization,
                                    max_tokens = max_features,
                                    output_mode = 'int',
                                    output_sequence_length = sequence_length)

In [23]:
train_text = raw_train_ds.map(lambda x, y:x)
vectorize_layer.adapt(train_text)

In [24]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [25]:
text_batch, label_batch = next(iter(raw_train_ds))
first_text, first_label = text_batch[0], label_batch[0]
print('first question ', first_text)
print('first label ', first_label)
print('vectorized question ', vectorize_text(first_text, first_label))

first question  tf.Tensor(b'"set blank to quit on exception? i\'m using blank 3..i\'ve been looking around for an answer to this, but i haven\'t found it yet. basically, i\'m running several blank scripts into a game engine, and each script has its own entry point...i\'d rather not add try: except blocks through all of my code, so i was wondering if it\'s at all possible to tell blank to quit (or perhaps assign a custom function to that ""callback"") on finding its first error, regardless of where or what it found? ..currently, the game engine will continue after finding and hitting an error, making it more difficult than necessary to diagnose issues since running into one error may make a subsequent script not work (as it relies on variables that the error-ing script set, for example). any ideas? ..i know that i could redirect the console to a file to allow for easier scrolling, but just capturing the first error and stopping the game prematurely would be really useful...okay, a coupl

In [26]:
print('17 ----> ', vectorize_layer.get_vocabulary()[17] )

17 ---->  blank


In [29]:
train_ds = raw_train_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)

In [30]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size = AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size = AUTOTUNE)

In [40]:
embedding_size = 16

model = tf.keras.Sequential([layers.Embedding(max_features+1, embedding_size),
                            layers.Dropout(0.2),
                            layers.GlobalAveragePooling1D(),
                            layers.Dropout(0.2),
                            layers.Dense(4)])

In [41]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 16)          160016    
_________________________________________________________________
dropout_4 (Dropout)          (None, None, 16)          0         
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 68        
Total params: 160,084
Trainable params: 160,084
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [45]:
epochs = 100

In [46]:
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/100
200/200 [==============================] - 1s 6ms/step - loss: 0.5841 - binary_accuracy: 0.2734 - val_loss: 0.6108 - val_binary_accuracy: 0.2672
Epoch 2/100
200/200 [==============================] - 1s 6ms/step - loss: 0.5472 - binary_accuracy: 0.2728 - val_loss: 0.5836 - val_binary_accuracy: 0.2663
Epoch 3/100
200/200 [==============================] - 1s 6ms/step - loss: 0.5159 - binary_accuracy: 0.2744 - val_loss: 0.5594 - val_binary_accuracy: 0.2663
Epoch 4/100
200/200 [==============================] - 1s 6ms/step - loss: 0.4859 - binary_accuracy: 0.2727 - val_loss: 0.5390 - val_binary_accuracy: 0.2648
Epoch 5/100
200/200 [==============================] - 1s 6ms/step - loss: 0.4629 - binary_accuracy: 0.2732 - val_loss: 0.5207 - val_binary_accuracy: 0.2652
Epoch 6/100
200/200 [==============================] - 1s 6ms/step - loss: 0.4411 - binary_accuracy: 0.2707 - val_loss: 0.5048 - val_binary_accuracy: 0.2639
Epoch 7/100
200/200 [==============================] - 1s 